In [25]:
import efel
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import scipy
import funcs_for_ck2_sk_study as ephys
from scipy.signal import find_peaks
from scipy import signal
from scipy import stats
from scipy.optimize import curve_fit

In [27]:
cc_path = os.getcwd()

In [28]:
#these code are well documented on efels github page. With strong documentation. These code are only used for the evoked spikes. All spontaneous firing code can be found in the "Spontaenous firing analysis" folder. Here we use a steeper threshold for the detection of spikes. 
#these code return excel sheets that will be saved in the path that you are currently in with all of the data. I have included a few example files to show you how to use the code.
os.chdir(cc_path)
f_i = ephys.analyze_feature(cc_path, feature='Spikecount_stimint')

Working on file:  211227 005.axgd
Working on file:  211227 009.axgd
Working on file:  211227 015.axgd
Working on file:  211227 023.axgd
Working on file:  220214 012.axgd


In [29]:
f_i

,211227 005.axgd,211227 009.axgd,211227 015.axgd,211227 023.axgd,220214 012.axgd
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
56,4,7,10,5,4
57,5,5,11,6,3
58,4,6,11,4,3
59,6,5,11,2,4
